In [10]:
import os
import CONFIG
from pprint import pprint
from langchain.vectorstores.chroma import Chroma
from langchain.embeddings import OpenAIEmbeddings
from langchain.chat_models import ChatOpenAI
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate
from pydantic import BaseModel, Field
from langchain.utils.openai_functions import convert_pydantic_to_openai_function



os.environ["OPENAI_API_KEY"] = CONFIG.OPENAI_API_KEY
llm_name = "gpt-4-1106-preview"

In [11]:
persist_directory = "/Users/dardonacci/Documents/dardonacci/2-Code/metabrain/vector_stores/hummingbot/chroma/"

embedding = OpenAIEmbeddings()

vectordb = Chroma(
    collection_name="hummingbot_scripts",
    persist_directory=persist_directory,
    embedding_function=embedding
)

In [33]:
from langchain.schema import StrOutputParser
from langchain.prompts import ChatPromptTemplate

# Build prompt
template = """Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer. You need to create the full code for the script that the user is asking for. If you think that more information is needed, ask the user for it.
Also remember this basic rules:
- The script should be written in Python.
- The variables to configure the script are defined as class variables.
- The script inherits the class ScriptStrategyBase.
- Each controller is initialized when defining the class variables and the executor handler is started in the init method.
{context}
Question: {question}
Helpful Answer:"""

prompt = ChatPromptTemplate.from_template(template=template)
model = ChatOpenAI(model_name=llm_name, temperature=0)
output_parser = StrOutputParser()

chain = prompt | model

In [34]:
query = """I would like to create a Directional Strategy using Hummingbot using the MACDBB controller. Can you the script for me and suggest values for the indicators?
"""

In [35]:
chain.invoke(
    input={
        "question": query,
    }
)

KeyError: 'context'

In [36]:
from langchain.schema.runnable import RunnableMap

retriever = vectordb.as_retriever(search_type="mmr", k=5, fetch_k=10)

chain = RunnableMap({
    "context": lambda x: retriever.get_relevant_documents(x["question"]),
    "question": lambda x: x["question"]
}) | prompt | model | output_parser

In [37]:
chain.invoke(
    input={
        "question": query,
    }
)

'Certainly! Below is a Python script for a Directional Strategy using Hummingbot with the MACDBB controller. The script inherits from `ScriptStrategyBase` and initializes the MACDBB controller with suggested values for the MACD and Bollinger Bands indicators. Please note that the values for the indicators are just suggestions and should be fine-tuned based on your trading preferences and backtesting results.\n\n```python\nfrom decimal import Decimal\nfrom hummingbot.connector.connector_base import ConnectorBase\nfrom hummingbot.smart_components.controllers.macd_bb_v1 import MACDBBV1, MACDBBV1Config\nfrom hummingbot.smart_components.strategy_frameworks.directional_trading.directional_trading_executor_handler import (\n    DirectionalTradingExecutorHandler,\n)\nfrom hummingbot.strategy.script_strategy_base import ScriptStrategyBase\n\n\nclass DirectionalMACDBBStrategy(ScriptStrategyBase):\n    # Configuration variables\n    trading_pair = "ETH-USDT"\n    exchange = "kucoin_paper_trade"\n

In [38]:
class Strategy(BaseModel):
    """Strategy created by the LLM for the user."""
    code: str = Field(description="Provide the python code for the strategy.")
    summary: str = Field(description="Provide a concise summary of what the strategy is doing.")

In [39]:

strategy_extraction_function = [
    convert_pydantic_to_openai_function(Strategy)
]
tagging_model = model.bind(
    functions=strategy_extraction_function,
    function_call={"name":"Strategy"}
)

In [40]:
from langchain.output_parsers.openai_functions import JsonOutputFunctionsParser

tagging_chain = RunnableMap({
    "context": lambda x: retriever.get_relevant_documents(x["question"]),
    "question": lambda x: x["question"]
}) | prompt | model | StrOutputParser() | tagging_model | JsonOutputFunctionsParser()

In [41]:
result = tagging_chain.invoke(
    input={
        "question": query,
    }
)

In [42]:
print(result["summary"])

This Python script defines a `DirectionalMACDStrategy` class for use with Hummingbot, which employs a MACDBB controller to inform trading decisions. The strategy is configured for the ETH-USDT trading pair on a paper trade exchange (Kucoin). It initializes the MACDBB controller with specific periods for the MACD and Bollinger Bands indicators, which can be adjusted to suit different trading styles and market conditions. The strategy includes methods for starting executor handlers, handling ticks, stopping the strategy, and formatting status updates. It is designed to be instantiated with the necessary connectors for the specified exchange.


In [43]:
print(result["code"])

from decimal import Decimal
from typing import Dict

from hummingbot.connector.connector_base import ConnectorBase
from hummingbot.core.data_type.common import OrderType
from hummingbot.data_feed.candles_feed.candles_factory import CandlesConfig
from hummingbot.smart_components.controllers.macd_bb_v1 import MACDBBV1, MACDBBV1Config
from hummingbot.smart_components.strategy_frameworks.directional_trading.directional_trading_executor_handler import (
    DirectionalTradingExecutorHandler,
)
from hummingbot.strategy.script_strategy_base import ScriptStrategyBase


class DirectionalMACDStrategy(ScriptStrategyBase):
    # Configuration
    trading_pair = "ETH-USDT"
    exchange = "kucoin_paper_trade"
    
    # MACDBB configuration
    macd_fast_period = 12
    macd_slow_period = 26
    macd_signal_period = 9
    bb_period = 20
    bb_std_dev_multiplier = 2
    
    # Initialize controllers and executor handler
    controllers = {
        trading_pair: MACDBBV1(
            config=MACDBBV1C